In [ ]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
from networkx.algorithms.coloring import greedy_color
import matplotlib.pyplot as plt
from copy import deepcopy

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch, get_window
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from scipy.stats import wasserstein_distance_nd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

In [ ]:
# settings
fs = 8000  # sampling frequency
f0 = 2000  # signal frequency
duration = 60  # seconds
snr_range = (-30, -45)
snr_vec = np.arange(snr_range[0], snr_range[1] - 1, -1)
num_thresholds = 10
num_iterations = 10

# welch args
nperseg = 1024
noverlap = 0.5
window = 'hanning'
dc = 20
crop_freq = None
norm_size = 5

# s2g args
quantization_bins = 10


In [ ]:
SNR = -30
welch_threshold = 0.04
s2g_threshold_wasserstein = 0.8
s2g_threshold_edge_count = 0.2
s2g_threshold_laplacian = 0.5

raw_signal = ut.simulate_raw_signal(f0, fs, duration)
F, T, raw_Sxx, raw_phasogram = ut.calc_spectrogram(raw_signal, fs, nperseg=nperseg, percent_overlap=noverlap/nperseg, window=window, remove_dc=dc, crop_freq=crop_freq)

fig = make_subplots(rows=6, cols=1, shared_xaxes=False, column_titles=[f"{SNR} dB"])
fig.add_trace(go.Heatmap(z=raw_Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=1)

rx = ut.add_noise_to_signal(raw_signal, snr_db=SNR, noise_type='white')
F, T, rx_Sxx, rx_phasogram = ut.calc_spectrogram(rx, fs, nperseg=nperseg, percent_overlap=noverlap, window=window, remove_dc=dc, crop_freq=crop_freq)
rx_pxx = ut.calc_welch_from_spectrogram(rx_Sxx, normalization_window_size=norm_size)
K = ut.get_all_Ks(rx_phasogram, F, n_levels=quantization_bins)

fig.add_trace(go.Heatmap(z=rx_Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=2, col=1)
fig.add_trace(go.Scatter(y=rx_pxx, x=np.arange(len(F)), showlegend=False, line=dict(color='blue')), row=3, col=1)

welch_detector = ut.WelchDetector(fs, nperseg, noverlap, window, dc, crop_freq, norm_size)
_, welch_detections = welch_detector.detect(rx, threshold=welch_threshold)
if len(welch_detections) > 0:
    fig.add_trace(go.Scatter(y=rx_pxx[welch_detections], x=np.arange(len(F))[welch_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=3, col=1)

s2g_detector = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="wasserstein")
_, s2g_detections, K = s2g_detector.detect(rx, threshold=s2g_threshold_wasserstein)
fig.add_trace(go.Scatter(y=K, x=np.arange(len(F)), showlegend=False, line=dict(color='purple')), row=4, col=1)
if len(s2g_detections) > 0:
    fig.add_trace(go.Scatter(y=K[s2g_detections], x=np.arange(len(F))[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=4, col=1)

s2g_detector = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="edge_count")
_, s2g_detections, K = s2g_detector.detect(rx, threshold=s2g_threshold_edge_count)
fig.add_trace(go.Scatter(y=K, x=np.arange(len(F)), showlegend=False, line=dict(color='purple')), row=5, col=1)
if len(s2g_detections) > 0:
    fig.add_trace(go.Scatter(y=K[s2g_detections], x=np.arange(len(F))[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=5, col=1)

s2g_detector = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="laplacian")
_, s2g_detections, K = s2g_detector.detect(rx, threshold=s2g_threshold_laplacian)
fig.add_trace(go.Scatter(y=K, x=np.arange(len(F)), showlegend=False, line=dict(color='purple')), row=6, col=1)
if len(s2g_detections) > 0:
    fig.add_trace(go.Scatter(y=K[s2g_detections], x=np.arange(len(F))[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=6, col=1)

fig.update_layout(height=800, width=800, title_text=f"Raw vs Noisy Signal")
fig.show()

In [ ]:
raw_signal = ut.simulate_raw_signal(f0, fs, duration)
F, T, raw_Sxx, raw_phasogram = ut.calc_spectrogram(raw_signal, fs, nperseg=nperseg, percent_overlap=noverlap/nperseg, window=window, remove_dc=dc, crop_freq=crop_freq)

SNR = snr_vec
welch_threshold = 0.04
s2g_threshold_wasserstein = 0.8
s2g_threshold_edge_count = 0.1
s2g_threshold_laplacian = 0.8

s2g_detector_wasserstein = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="wasserstein")
s2g_detector_edge_count = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="edge_count")
s2g_detector_laplacian = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="laplacian")


_noise_greneration_time = []
_spectrogram_time = []
_welch_time = []
_welch_detection_time = []
_s2g_detection_time_wass = []
_s2g_detection_time_edge = []
_s2g_detection_time_lap = []

fig = make_subplots(rows=6, cols=len(SNR), shared_xaxes=False, column_titles=[f"{snr} dB" for snr in SNR])
for i, snr in enumerate(SNR, start=1):
    _t = time.time()
    rx = ut.add_noise_to_signal(raw_signal, snr_db=snr, noise_type='white')
    _noise_greneration_time.append(time.time() - _t)
    fig.add_trace(go.Heatmap(z=raw_Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=i)

    _t = time.time()
    _F, _T, rx_Sxx, _phasogram = ut.calc_spectrogram(rx, fs, nperseg=nperseg, percent_overlap=noverlap, window=window, remove_dc=dc, crop_freq=crop_freq)
    _spectrogram_time.append(time.time() - _t)
    fig.add_trace(go.Heatmap(z=rx_Sxx, x=_T, y=_F, colorscale='Viridis', showlegend=False, showscale=False), row=2, col=i)

    _t = time.time()
    rx_pxx = ut.calc_welch_from_spectrogram(rx_Sxx, normalization_window_size=norm_size)
    _welch_time.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=rx_pxx, x=_F, showlegend=False, line=dict(color='blue')), row=3, col=i)
    
    _t = time.time()
    _, welch_detections = welch_detector.detect(rx, threshold=welch_threshold)
    _welch_detection_time.append(time.time() - _t)
    if len(welch_detections) > 0:
        fig.add_trace(go.Scatter(y=rx_pxx[welch_detections], x=_F[welch_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=3, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_wasserstein.detect(rx, threshold=s2g_threshold_wasserstein)
    _s2g_detection_time_wass.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=4, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=4, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_edge_count.detect(rx, threshold=s2g_threshold_edge_count)
    _s2g_detection_time_edge.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=5, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=5, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_laplacian.detect(rx, threshold=s2g_threshold_laplacian)
    _s2g_detection_time_lap.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=6, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=6, col=i)


print("Noise generation times:", np.average(_noise_greneration_time))
print("Spectrogram calculation times:", np.average(_spectrogram_time))
print("Welch calculation times:", np.average(_welch_time))
print("Welch detection times:", np.average(_welch_detection_time))
print("S2G detection times (Wasserstein):", np.average(_s2g_detection_time_wass))
print("S2G detection times (Edge Count):", np.average(_s2g_detection_time_edge))
print("S2G detection times (Laplacian):", np.average(_s2g_detection_time_lap))

fig.update_layout(height=800, width=300*len(SNR), title_text=f"Raw vs Noisy Signal")
fig.show()